In [1]:
path_data = '../../../../data/'
import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from urllib.request import urlopen
import re
from bancor_simulator.protocol import BancorNetwork
from bancor_simulator.core.settings import GlobalSettings
from decimal import Decimal
from fractions import Fraction
import pandas as pd

# Recall from an earlier chapter that we already defined the cooldown period and the exit fees as follows.
cooldown_time: int = 7
withdrawal_fee: Decimal = Decimal('0.0025')
whitelisted_tokens: list = ['bnt', 'eth', 'wbtc', 'link']
bnt_funding_limit: Decimal = Decimal('40000')
bnt_min_liquidity: Decimal = Decimal('10000')
price_feeds = pd.DataFrame({'INDX':[0],
                            'bnt':[2.5],
                            'link':[15.00],
                            'eth':[2500.00],
                            'wbtc':[40000.00]})

network_fee: Decimal = Decimal('0.2')
trading_fee: Decimal = Decimal('0.01')

# There are other possible configuration settings available, however for the present purpose we will use the defaults.
v3 = BancorNetwork(whitelisted_tokens=whitelisted_tokens,
              network_fee=network_fee,
              trading_fee=trading_fee,
              cooldown_time=cooldown_time,
              withdrawal_fee=withdrawal_fee,
              bnt_min_liquidity=bnt_min_liquidity,
              bnt_funding_limit=bnt_funding_limit,
              price_feeds=price_feeds)

v3.create_user('Alice')
v3.create_user('Bob')
v3.create_user('Charlie')

# The actual deposits.
v3.stake(tkn_amt=100, tkn_name='eth', user_name='Alice')

v3.stake(tkn_amt=10000, tkn_name='link', user_name='Charlie')

v3.stake(tkn_amt=100, tkn_name='wbtc', user_name='Bob')

v3.stake(tkn_amt=1, tkn_name='eth', user_name='Alice')

v3.stake(tkn_amt=1, tkn_name='link', user_name='Charlie')

v3.stake(tkn_amt=1, tkn_name='wbtc', user_name='Bob')

v3.dao_msig_init_pools(whitelisted_tokens)

v3.create_user('Trader')

v3.trade(swap_amount=2000, source_token='bnt',  target_token='link', user_name='Trader', timestep=0)

v3.trade(swap_amount=302.9981, source_token='link',  target_token='bnt', user_name='Trader', timestep=0)

v3.trade(swap_amount=1, source_token='eth',  target_token='wbtc', user_name='Trader', timestep=0)

# v3.global_state.vault_bnt =

def read_url(url): 
    return re.sub('\\s+', ' ', urlopen(url).read().decode())

# Staking BNT

In [2]:
v3.describe(decimals=4)

,Trading Liquidity,Vault,Staking,ERC20 Contracts,Vortex,External Protection,Protocol WalletState
0,,bnt=60022.3847,bnt=60023.5998,bnbnt=60000.0000,bnt=12.2692,bnt=0.0000,bnbnt=60000.0000
1,bnt=19055.2381 eth=21.0000,eth=102.0000,eth=101.0000,bneth=101.0000,,eth=0.0000,
2,bnt=20940.8835 wbtc=1.1943,wbtc=100.9443,wbtc=101.0005,bnwbtc=101.0000,,wbtc=0.0000,
3,bnt=20013.9939 link=3336.3314,link=10003.9981,link=10003.4242,bnlink=10001.0000,,link=0.0000,
4,,,,,,,


In [3]:
v3.create_user('David')

timestep = v3.global_state.tokens['bnt'].timestep
print('timestep', timestep)
v3.stake(tkn_amt=1000, tkn_name='bnt',  user_name='David', timestep=timestep)

v3.describe(decimals=4)

timestep 0


,Trading Liquidity,Vault,Staking,ERC20 Contracts,Vortex,External Protection,Protocol WalletState
0,,bnt=60022.3847,bnt=60023.5998,bnbnt=60000.0000,bnt=12.2692,bnt=0.0000,bnbnt=59000.0000
1,bnt=19055.2381 eth=21.0000,eth=102.0000,eth=101.0000,bneth=101.0000,,eth=0.0000,
2,bnt=20940.8835 wbtc=1.1943,wbtc=100.9443,wbtc=101.0005,bnwbtc=101.0000,,wbtc=0.0000,
3,bnt=20013.9939 link=3336.3314,link=10003.9981,link=10003.4242,bnlink=10001.0000,,link=0.0000,
4,,,,,,,


# Staking BNT

In Bancor 3, assume that BNT liquidity providers are receiving protocol bnBNT tokens in return for destroying BNT. The calculation is essentially identical to the standard pool token method, and its outcome is relatively easy to understand. However, there are some nuances to the process that should be highlighted.

For this section, we can consider the state of the system after the initial bootstrapping and first trading activity described earlier. The system snapshot is as follows:


In [4]:
timestep = v3.global_state.tokens['wbtc'].timestep
print('timestep', timestep)
v3.stake(tkn_amt=1, tkn_name='eth',  user_name='Alice', timestep=timestep)

v3.describe(decimals=4)

timestep 0


,Trading Liquidity,Vault,Staking,ERC20 Contracts,Vortex,External Protection,Protocol WalletState
0,,bnt=60022.3847,bnt=60023.5998,bnbnt=60000.0000,bnt=12.2692,bnt=0.0000,bnbnt=59000.0000
1,bnt=19055.2381 eth=21.0000,eth=103.0000,eth=102.0000,bneth=102.0000,,eth=0.0000,
2,bnt=20940.8835 wbtc=1.1943,wbtc=100.9443,wbtc=101.0005,bnwbtc=101.0000,,wbtc=0.0000,
3,bnt=20013.9939 link=3336.3314,link=10003.9981,link=10003.4242,bnlink=10001.0000,,link=0.0000,
4,,,,,,,


In [5]:
v3.describe(rates=True, decimals=4)

,index,0
0,bnt,"Spot Rate=0.0000, EMA Rate=0.0000"
1,eth,"Spot Rate=907.3923, EMA Rate=1000.0000"
2,wbtc,"Spot Rate=17534.2111, EMA Rate=16000.0000"
3,link,"Spot Rate=5.9988, EMA Rate=6.0000"


The most important nuance to be aware of is that BNT liquidity providers are supporting the BNT liquidity of the whole protocol, rather than any specific pool. As a result, there is no spot price or moving average. Further, bnBNT pool tokens are not created when a BNT liquidity provider adds their tokens to the protocol; there is no change in the vault balance of BNT, or its staked balance (save for one extreme edge case). The provision of BNT by users is best thought of as a private exchange of BNT directly for protocol-owned bnBNT.

To demonstrate, assume a fourth participant, David, wishes to provide 1,000 BNT liquidity to the protocol. The only calculations the protocol must perform are to value the BNT David is providing. The staking ledger is reporting a total of 6,002.3599 BNT, and the bnBNT pool token supply is 6,000 bnBNT. Therefore, the bnBNT/BNT exchange rate is 0.9996068, and David’s 1,000 BNT is worth 999.60683797 bnBNT. When David confirms this transaction, the protocol simply transfers this amount of bnBNT to him, from its own balance; the BNT that David provided is burned immediately.

Note that the only observable change in the system is that the protocol owns less of the total BNT; outside of the system, the ERC20 contract supply of BNT will have diminished slightly. David also receives vBNT, the Bancor governance token, at a 1:1 rate with respect to the bnBNT pool tokens (see the following section). vBNT is staked in the governance contract to take part in Bancor’s decision making process; however, it must also be returned when a user exits from the protocol.
